<a href="https://colab.research.google.com/github/shahaman06/CMPE297-ShortStory/blob/main/Colab_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Intial Files Setup

!rm -r *
!git clone "https://github.com/shahaman06/CMPE297-ShortStory/"
!mv /content/CMPE297-ShortStory/* /content/
!rm -r CMPE297-ShortStory README.md Colab_Notebook.ipynb

Cloning into 'CMPE297-ShortStory'...
remote: Enumerating objects: 778, done.
remote: Counting objects: 100% (778/778), done.
remote: Compressing objects: 100% (776/776), done.
remote: Total 778 (delta 6), reused 773 (delta 2), pack-reused 0
Receiving objects: 100% (778/778), 10.61 MiB | 12.87 MiB/s, done.
Resolving deltas: 100% (6/6), done.


In [85]:
# import torch.nn as nn
# import torchvision
import os
import cv2
import numpy as np

from torch.cuda import is_available
from torch.utils.data import DataLoader,Dataset
from torch.optim import Adam
from torch.nn import BCELoss
from torch import ones, randn, tensor
from torchvision import transforms
from modules import Discriminator, Generator, PaintingDataset
from tqdm import tqdm
from glob import glob

device = 'cuda' if is_available() else 'cpu'

In [86]:
# Declaring GLOBAL VARS:
BATCH_SIZE = 5
IMG_SHAPE = 208
CHANNELS = 3
EPOCHS = 5
FEATURES = 16
RANDOM_NOISE_SHAPE = 256

In [87]:
class PaintingDataset(Dataset):
    def __init__(self, loc = os.path.join('/content/paintings/'), img_shape = 208):
        self.loc = loc
        self.paintings = glob(os.path.join(loc+'*.jpg'))
        self.img_shape = img_shape
        print(self.paintings)
        
    def __len__(self):
        return len(self.paintings)
    
    def __getitem__(self, idx):
        img = self.paintings[idx]
        img = cv2.imread(img)
        img = cv2.resize(img, (self.img_shape, self.img_shape))
        img = np.moveaxis(img, -1, 0) # pytorch takes channel first images
        img = tensor(img).float()
        return img.to(device)

In [88]:
ds = PaintingDataset(img_shape = IMG_SHAPE,)
ds[1].shape # -> torch.Size([3, 208, 208])
loader = DataLoader(ds, shuffle = True, batch_size = BATCH_SIZE)
batch = next(iter(loader))
batch.shape # -> torch.Size([5,3, 208, 208])

['/content/paintings/1999.jpg', '/content/paintings/0472.jpg', '/content/paintings/2123.jpg', '/content/paintings/2086 20.26.31.jpg', '/content/paintings/1671.jpg', '/content/paintings/2281.jpg', '/content/paintings/2303.jpg', '/content/paintings/0328.jpg', '/content/paintings/0819.jpg', '/content/paintings/0608.jpg', '/content/paintings/0220.jpg', '/content/paintings/0642.jpg', '/content/paintings/1912.jpg', '/content/paintings/0376.jpg', '/content/paintings/1526.jpg', '/content/paintings/1137.jpg', '/content/paintings/0072.jpg', '/content/paintings/0840.jpg', '/content/paintings/1227.jpg', '/content/paintings/0389.jpg', '/content/paintings/1122.jpg', '/content/paintings/0431.jpg', '/content/paintings/1983.jpg', '/content/paintings/1447.jpg', '/content/paintings/1628.jpg', '/content/paintings/1078.jpg', '/content/paintings/0244.jpg', '/content/paintings/2081.jpg', '/content/paintings/1278.jpg', '/content/paintings/2236.jpg', '/content/paintings/2251.jpg', '/content/paintings/2076.jpg'

torch.Size([5, 3, 208, 208])

In [ ]:
disc = Discriminator().to(device)
gen = Generator().to(device)

lr = 0.0002
epochs = 2

optimD = Adam(disc.parameters(), lr = lr, betas = (0.5, 0.99))
optimG = Adam(gen.parameters(), lr = lr, betas = (0.5, 0.99))

criterion = BCELoss()

real_label = 1
fake_label = 0

# fixed_noise = randn(batch_size, channels_noise, 1, 1).to(device)

for epoch in tqdm(range(epochs)):
    for data in loader:
        batch_size = data.shape[0]

        ## Training discriminator

        '''First feeding the real images'''
        label = (ones(batch_size)* 0.9).to(device)
        output = disc(data).reshape(-1)
        lossD_real = criterion(output, label)

        '''feeding generated images'''
        label = (ones(batch_size) * 0.1).to(device)
        rand_noise = randn((batch_size, RANDOM_NOISE_SHAPE, 1, 1)).to(device)
        fake_image = gen(rand_noise)
        output = disc(fake_image.detach()).reshape(-1)
        lossD_fake = criterion(output, label)

        '''Back propogating discriminator and updating weights'''
        disc.zero_grad()
        lossD = lossD_real + lossD_fake
        lossD.backward()
        optimD.step()

        ## Training generator

        label = ones(batch_size).to(device)
        output = disc(fake_image).reshape(-1)
        lossG = criterion(output, label)

        '''Backpropogating'''
        gen.zero_grad()
        lossG.backward()
        optimG.step()